In [ ]:
!pip install torchvision
!pip install pycocotools
!pip install opencv-python
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━ 181.2/363.4 MB 20.4 MB/s eta 0:00:09
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.2 MB/s eta 0:00:00
ERROR: THESE PACKAGES DO NOT MATCH THE HASHES FROM THE REQUIREMENTS FILE. If you have updated the package versions, please update the hashes. Otherwise, examine the package contents carefully; someone ma

In [ ]:
!pip install pyyaml==5.1
!pip install torch torchvision torchaudio
!pip install git+https://github.com/facebookresearch/detectron2.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 12.9 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.structures import BoxMode
import cv2
import numpy as np
import os
import json
from PIL import Image
from tqdm import tqdm

In [ ]:
!unrar x "/content/test.rar" "/content/images_unzipped/"


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/test.rar


Would you like to replace the existing file /content/images_unzipped/test/L12.jpg
257444 bytes, modified on 2022-02-02 02:21
with a new one
257444 bytes, modified on 2022-02-02 02:21

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit n


Would you like to replace the existing file /content/images_unzipped/test/L13 (2).jpg
242168 bytes, modified on 2022-02-04 05:58
with a new one
242168 bytes, modified on 2022-02-04 05:58

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit 
User break

User break


In [ ]:
# ======== CONFIGURATION ========
image_dir = "/content/images_unzipped/test/"
output_json_path = "/content/output_annotations_coco.json"
model_path = "/content/drive/MyDrive/Project Akhir/model_final (1).pth"
class_names = ["imapksi", "karies", "lesi periapikal"]
confidence_threshold = 0.7

# ======== DETECTRON2 SETUP ========
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.NUM_CLASSES = len(class_names)
cfg.MODEL.WEIGHTS = model_path
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = confidence_threshold
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

predictor = DefaultPredictor(cfg)

# Register custom metadata
MetadataCatalog.get("custom_dataset").thing_classes = class_names

# ======== MAIN LOOP ========
images = sorted([f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))])

coco_output = {
    "images": [],
    "annotations": [],
    "categories": [{"id": i + 1, "name": name} for i, name in enumerate(class_names)]
}

annotation_id = 1
for image_id, img_name in tqdm(enumerate(images), total=len(images)):
    img_path = os.path.join(image_dir, img_name)
    image = cv2.imread(img_path)
    height, width = image.shape[:2]

    outputs = predictor(image)
    instances = outputs["instances"].to("cpu")

    # Register image
    coco_output["images"].append({
        "id": image_id,
        "file_name": img_name,
        "width": width,
        "height": height
    })

    for i in range(len(instances)):
        mask = instances.pred_masks[i].numpy().astype(np.uint8)
        label = int(instances.pred_classes[i]) + 1
        score = float(instances.scores[i])

        if score < confidence_threshold:
            continue

        # Extract contour and segmentation
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        segmentation = []
        for contour in contours:
            if len(contour) >= 6:
                segmentation.append(contour.flatten().tolist())

        if not segmentation:
            continue

        from pycocotools import mask as mask_utils
        rle = mask_utils.encode(np.asfortranarray(mask))
        area = float(mask_utils.area(rle))
        bbox = mask_utils.toBbox(rle).tolist()

        coco_output["annotations"].append({
            "id": annotation_id,
            "image_id": image_id,
            "category_id": label,
            "segmentation": segmentation,
            "area": area,
            "bbox": bbox,
            "iscrowd": 0
        })
        annotation_id += 1

# ======== SAVE OUTPUT ========
with open(output_json_path, "w") as f:
    json.dump(coco_output, f)

print("Saved:", output_json_path)


  0%|          | 0/43 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 43/43 [06:04<00:00,  8.48s/it]

Done! COCO JSON saved at: /content/output_annotations_coco.json
